In [2]:
import liana as li

In [5]:
import scanpy as sc

In [8]:
import numpy as np

In [3]:
li.method.show_methods()

,Method Name,Magnitude Score,Specificity Score,Reference
0,CellPhoneDB,lr_means,cellphone_pvals,"Efremova, M., Vento-Tormo, M., Teichmann, S.A...."
0,Connectome,expr_prod,scaled_weight,"Raredon, M.S.B., Yang, J., Garritano, J., Wang..."
0,log2FC,None,lr_logfc,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,NATMI,expr_prod,spec_weight,"Hou, R., Denisenko, E., Ong, H.T., Ramilowski,..."
0,SingleCellSignalR,lrscore,None,"Cabello-Aguilar, S., Alame, M., Kon-Sun-Tack, ..."
0,Rank_Aggregate,magnitude_rank,specificity_rank,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,Geometric Mean,lr_gmeans,gmean_pvals,CellPhoneDBv2's permutation approach applied t...
0,scSeqComm,inter_score,None,"Baruzzo, G., Cesaro, G., Di Camillo, B. 2022. ..."
0,CellChat,lr_probs,cellchat_pvals,"Jin, S., Guerrero-Juarez, C.F., Zhang, L., Cha..."


In [4]:
lr_pairs = li.resource.select_resource('consensus')

In [6]:
adata = sc.read_h5ad("/home/igarzonalva/Proyecto_SC_TNBC/GSE161529/LabelTransfer/adatas/full_adata_post_label_transfer.h5ad")

In [13]:
fraction = 0.1
n_cells = int(fraction * adata.n_obs)
random_indices = np.random.choice(adata.n_obs, size=n_cells, replace=False)

In [14]:
random_indices

array([151340, 112336,  15565, ...,  11532,  36393,  56170])

In [15]:
adata_sub = adata[random_indices, :].copy()

In [16]:
adata_sub

AnnData object with n_obs × n_vars = 15231 × 18088
    obs: 'sample', 'n_genes_by_counts', 'total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'pct_counts_mt', 'complexity', 'doublet_scores', 'predicted_doublets', 'n_genes', 'n_counts', 'celltypist_labels_Immune_All_High', 'celltypist_scores_Immune_All_High', 'celltypist_labels_Cells_Adult_Breast', 'celltypist_scores_Cells_Adult_Breast', 'batch', 'subtype', 'leiden', 'celltypist_labels_Immune_All_High_highconf', 'GenAnno', 'GenAnnoV1', 'GenAnnoV2', 'celltypist_mapped', 'GenAnnoHighConf', 'cell_type', 'predicted_labels', 'GennAnno_ScAnvi', 'LabelTransferAnno'
    var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'GenAnnoHighConf_colors', 'GenAnnoV1_colors', 'GenAnnoV2_colors', 'GennAnno_ScAnvi_colors', 'LabelTransferAnno_colors', 'celltypist_labels_Immune_All_

In [18]:
li.mt.rank_aggregate.by_sample(adata,
                               sample_key='batch',
                               groupby='LabelTransferAnno',
                               resource_name = 'consensus',
                               expr_prop=0.1, # must be expressed in expr_prop fraction of cells
                               min_cells = 5,
                               n_perms = 100,
                               use_raw = False, # run on log- and library-normalized counts
                               verbose = True,
                               inplace = True
                              )

Now running: PM0337-:   0%|                                                   | 0/28 [00:00<?, ?it/s]/usr/local/lib/python3.10/site-packages/anndata/_core/anndata.py:402: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
/usr/local/lib/python3.10/site-packages/liana/method/_pipe_utils/_pre.py:150: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
/usr/local/lib/python3.10/site-packages/liana/method/_pipe_utils/_pre.py:153: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
/usr/local/lib/python3.10/site-packages/liana/method/sc/_liana_pipe.py:262: ImplicitModificationWarning: Setting element `.layers['scaled']` of view, initializing view as actual.
Now running: B1-Tum0554-:   4%|█▍                                     | 1/2

In [22]:
adata.uns['liana_res'].head()

,batch,source,target,ligand_complex,receptor_complex,lr_means,cellphone_pvals,expr_prod,scaled_weight,lr_logfc,spec_weight,lrscore,specificity_rank,magnitude_rank
0,PM0337-,mCAF,Pericyte,LUM,ITGB1,3.572700,0.0,11.824445,4.248750,4.347075,0.026958,0.953606,0.000040,4.843218e-10
1,PM0337-,mCAF,mCAF,LUM,ITGB1,3.522689,0.0,11.370135,4.182577,4.338816,0.025922,0.952731,0.000047,4.358853e-09
2,PM0337-,IDO_CAF,Mast_cells,TIMP1,CD63,3.456436,0.0,11.796284,2.525780,3.091284,0.004903,0.953553,0.011501,4.358853e-09
3,PM0337-,IDO_CAF,Pericyte,LUM,ITGB1,3.451137,0.0,11.191518,4.088944,3.710209,0.025515,0.952373,0.000057,7.749034e-09
4,PM0337-,mCAF,Pericyte,LGALS1,ITGB1,3.380153,0.0,10.821932,2.281900,2.730287,0.004822,0.951606,0.024897,3.099553e-08


In [24]:
adata.uns['liana_res'].columns

Index(['batch', 'source', 'target', 'ligand_complex', 'receptor_complex',
       'lr_means', 'cellphone_pvals', 'expr_prod', 'scaled_weight', 'lr_logfc',
       'spec_weight', 'lrscore', 'specificity_rank', 'magnitude_rank'],
      dtype='object')